In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np


def loss(true_value , prediction_value):
    true_value = np.array(true_value)
    prediction_value = np.array(prediction_value)

    MSE =  sum( (true_value - prediction_value)**2 ) / len(true_value)

    return MSE

def BIC(mse , n, j):
    bic = n * (np.log(mse)) + j* ( np.log(n) )
    return MSE

def AIC(mse , n, j):
    aic = n * (np.log(mse)) + 2*j
    return MSE

##Read dataset

In [2]:
## read data from drive
data_xlxs = pd.read_excel(open('/content/AirQualityUCI.xlsx', 'rb'))
data_df = pd.DataFrame(data_xlxs, columns=data_xlxs.columns[0:])

## Handle Missing values


> Note: Since all the features are not measured in the same scale , **NORMALIZING** all the features should be explored for data preprocessing part


In [3]:
air_df = data_df.iloc[0:, 0:]


air_df = air_df.astype({"Time": str})
# converting time to int for easier ploting
air_df['Time'] = pd.to_datetime(air_df['Time']).dt.hour

## drop feature with more then 50% Missing values
num_missing = (air_df == -200).sum()
drop_attributes=['Date'] ## Date object droped
attributes = air_df.columns.to_list()
for item in attributes:
    if num_missing[item] >= ( 0.5 * len(air_df[item])):
        drop_attributes.append(item)
air_df = air_df.drop(columns= drop_attributes)


## -200 indicates missing value, replace with random value around the mean of the column with std
air_df = air_df.replace(-200, np.nan )

attributes = air_df.columns.to_list()
for item in attributes:
    mean = air_df[item].mean()
    std = air_df[item].std()
    low= -std
    high= std
    rand = mean + np.random.uniform( low= -std, high= std )
    # print( mean, std, low, high, rand )
    air_df[item] = air_df[item].replace( np.nan, mean+rand )

num_missing = (air_df[item] == -200).sum()
print("Missing values per column :", num_missing)

# sns.pairplot(air_df)
# plt.rcParams['figure.figsize']=(10,10)

Missing values per column : 0


In [4]:
## rearrange the dataframe for C6H6 level prediction
columns_titles = ['Time', 'CO(GT)', 'PT08.S1(CO)', 'PT08.S2(NMHC)',
                  'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
                  'PT08.S5(O3)', 'T', 'RH', 'AH', 'C6H6(GT)']
air_df = air_df.reindex(columns=columns_titles)
attributes = air_df.columns.to_list()
# print(attributes)



X = air_df.iloc[: , 0:-1]
y = np.array( air_df.iloc[:, -1:] )
y = y.flatten()
# print(X.shape)
# print(y.shape)

air_df.head()

,Time,CO(GT),PT08.S1(CO),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,C6H6(GT)
0,18,2.6,1360.00,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754,11.881723
1,19,2.0,1292.25,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487,9.397165
2,20,2.2,1402.00,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239,8.997817
3,21,2.2,1375.50,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713,9.228796
4,22,1.6,1272.25,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794,6.518224


##PCA



In [5]:
from sklearn import decomposition
number_of_features = 6
pca = decomposition.PCA(n_components = number_of_features)
pca.fit(X)
X_transformed = pca.transform(X)
print(X_transformed.shape)

(9357, 6)


In [6]:
from sklearn import decomposition
number_of_features = 6
pca = decomposition.PCA(n_components = number_of_features)
pca.fit(X)
X_transformed = pca.transform(X)
print(X_transformed.shape)



from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X_transformed, y, test_size = 0.30, random_state = 1)
# print(x_train.shape)
# print(y_train.shape)
# print(x_test.shape)
# print(y_test.shape)

print(type(x_train))
print(type(y_train))
print(x_test.shape)
print(y_test.shape)

pca = decomposition.PCA(n_components= 1)
pca.fit(x_train)
x_train_SLR = pca.transform(x_train)
x_test_SLR = pca.transform(x_test)
# print(x_train_SLR.shape)
# print(y_train.shape)
# print(x_test_SLR.shape)
# print(y_test.shape)

(9357, 6)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(2808, 6)
(2808,)


In [ ]:
## model scores list
Models = []
MSE = []
R_sq = []

Bic = []
Ric = []

##Simple Linear Reg

In [ ]:
import statsmodels.api as sm

linear_mod = sm.OLS( y_train  , x_train_SLR)
res = linear_mod.fit()


y_pred = res.predict( x_test_SLR )
print("\nMSE", loss(y_test, y_pred) )
print("R_squared", res.rsquared)

print(res.summary())

Models.append('Simple Linear Reg.')
MSE.append( loss(y_test, y_pred) )
R_sq.append(res.rsquared ) 


print("\nModels:\n", Models)
print(MSE)
print(R_sq)



##Multiple Linear Reg

In [ ]:
multiple_linear_mod = sm.OLS( y_train  , x_train)
res = multiple_linear_mod.fit()

y_pred = res.predict( x_test )

print("\nMSE", loss(y_test, y_pred) )
print("R_squared", res.rsquared)
print(res.summary())

Models.append('Multiple Linear Reg.')
MSE.append( loss(y_test, y_pred) )
R_sq.append(res.rsquared )

print("\nModels:\n", Models)
print(MSE)
print(R_sq)

##Polynomial Reg

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(degree=2, include_bias=False)
transformer.fit(X_transformed)
x_train_PLR = transformer.transform(x_train)
# print(x_train_PLR.shape)

plynomial_reg_mod = sm.OLS( y_train  , x_train_PLR)
res = plynomial_reg_mod.fit()

x_test_PLR = transformer.transform(x_test)
y_pred = res.predict( x_test_PLR )

print("\nMSE", loss(y_test, y_pred) )
print("R_squared", res.rsquared)
print(res.summary())

Models.append('Polynomial Reg.')
MSE.append( loss(y_test, y_pred) )
R_sq.append(res.rsquared ) 

print("\nModels:\n", Models)
print(MSE)
print(R_sq)

## LASSO reg

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(x_train, y_train)

r_sq_lasso = lasso.score(x_train, y_train)


y_pred = lasso.predict( x_test )

print("\nMSE", loss(y_test, y_pred) )
print("R_squared", r_sq_lasso)

Models.append('LASSO Reg.')
MSE.append( loss(y_test, y_pred) )
R_sq.append( r_sq_lasso ) 

print("\nModels:\n", Models)
print(MSE)
print(R_sq)

##RIDGE

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(x_train, y_train)

r_sq_ridge = ridge.score(x_train, y_train)


y_pred = ridge.predict( x_test )
print("\nMSE", loss(y_test, y_pred) )
print("R_squared", r_sq_ridge)

Models.append('Ridge Reg.')
MSE.append( loss(y_test, y_pred) )
R_sq.append( r_sq_ridge ) 

print("\nModels:\n", Models)
print(MSE)
print(R_sq)

##SVR

In [ ]:
import numpy as np
from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf')
svr_rbf.fit(x_train, y_train)

r_sq_svr = svr_rbf.score(x_train, y_train)


y_pred = svr_rbf.predict( x_test )
print("\nMSE", loss(y_test, y_pred) )
print("R_squared", r_sq_svr)

Models.append('SVR')
MSE.append( loss(y_test, y_pred) )
R_sq.append( r_sq_svr ) 

print("\nModels:\n", Models)
print(MSE)
print(R_sq)

##K_fold SVR

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

k_fold = KFold(n_splits=10)

svr_rbf_k_fold = SVR(kernel='rbf')

count=0
for train_indices, test_indices in k_fold.split(x_train):
    count +=1
    print("\nK_fold: ",count)
    # print('Train: %s | test: %s' % (train_indices, test_indices))
    svr_rbf_k_fold.fit(x_train[train_indices], y_train[train_indices])
    svr_rbf_k_fold.score(x_train[test_indices], y_train[test_indices])

    y_pred = svr_rbf_k_fold.predict( x_train[test_indices] )
    print("\tMSE:", loss(y_train[test_indices], y_pred) )

    r_sq_svr_rbf_k_fold = svr_rbf_k_fold.score(x_train[train_indices], y_train[train_indices])
    print('\tR^2:', r_sq_svr_rbf_k_fold)


r_sq_svr_rbf_k_fold = svr_rbf_k_fold.score(x_train, y_train)


y_pred = svr_rbf_k_fold.predict( x_test )
print("\nMSE", loss(y_test, y_pred))
print('R_Squared', r_sq_svr_rbf_k_fold , "\n")

Models.append('SVR_K_fold')
MSE.append( loss(y_test, y_pred) )
R_sq.append( r_sq_svr_rbf_k_fold ) 

print("\nModels:\n", Models)
print("MSE scr\n", MSE)
print("R_squared\n",R_sq)

## Model compare

In [ ]:
plt.figure( figsize=(16, 8) )
plt.scatter(R_sq, MSE)

for i, txt in enumerate(Models):
    plt.annotate(txt, (R_sq[i], MSE[i]))

